In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")

In [3]:
test = pd.read_csv("/kaggle/input/playground-series-s4e7/test.csv")

In [4]:
vehicle_age = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years':2}

train['Vehicle_Age'] = train['Vehicle_Age'].map(vehicle_age)
test['Vehicle_Age'] = test['Vehicle_Age'].map(vehicle_age)

In [5]:
from sklearn.model_selection import train_test_split


X, y = train_test_split(train, stratify = train['Response'], test_size = 0.2, random_state = 42)

In [6]:
X_train = X.drop('Response', axis = 1)
y_train = X['Response']
X_test = y.drop('Response', axis = 1)
y_test = y['Response']


In [7]:
X = train.drop(['Response', 'id'], axis = 1)
y = train['Response']

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

num = ['Age', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Annual_Premium', 'Policy_Sales_Channel', 'Vintage']
cat = ['Gender', 'Vehicle_Damage']

num_pipeline = make_pipeline(StandardScaler())

cat_pipeline = make_pipeline(OrdinalEncoder())

preprocessing = ColumnTransformer([
    ("cat", cat_pipeline, cat),
    # ('both', StandardScaler(), X_train.columns.values)
    ],
    
    remainder = 'passthrough'
)

X_train_prep = preprocessing.fit_transform(X)

In [9]:
X_preprocessed = pd.DataFrame(X_train_prep, columns = preprocessing.get_feature_names_out(), index = train.index)

In [10]:
X_preprocessed.columns

Index(['cat__Gender', 'cat__Vehicle_Damage', 'remainder__Age',
       'remainder__Driving_License', 'remainder__Region_Code',
       'remainder__Previously_Insured', 'remainder__Vehicle_Age',
       'remainder__Annual_Premium', 'remainder__Policy_Sales_Channel',
       'remainder__Vintage'],
      dtype='object')

In [11]:
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from sklearn.ensemble import RandomForestClassifier

# params = {
#     'max_depth' : np.linspace(3,7, 5, dtype = np.uint8),
#     'max_leaf_nodes' : np.linspace(50,200,150, dtype = np.uint8),
#     'min_samples_split' : [2,3,4,5]
# }

# rnd_search = RandomizedSearchCV(
#     RandomForestClassifier(n_estimators = 10, random_state = 42, n_jobs = -1), params, n_iter = 60, cv = 3
# )

In [12]:
# rnd_search.fit(X_preprocessed, y)

In [13]:
# pipeline = make_pipeline(preprocessing, rnd_search.best_estimator_)

# pipeline.fit(X, y)

In [14]:
from sklearn.ensemble import RandomForestClassifier


rf_clf = RandomForestClassifier(max_depth=6, max_leaf_nodes=87, min_samples_split=4,
                       random_state=42, n_estimators = 300, n_jobs = -1)

In [15]:
pipeline = make_pipeline(preprocessing, rf_clf)
pipeline.fit(X, y)


ids = test['id']
test = test.drop('id', axis = 1)

In [16]:
y_pred = pipeline.predict_proba(test)

In [17]:
ans = pd.DataFrame({'id': ids, 'Response' : y_pred[:,1]})

In [18]:
ans.to_csv("Random Forest.csv", index = False)